In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats
import pickle

In [2]:
df_dcinside = pd.read_csv("./teamkim_datas.SNS_DCinside.csv")
df_helpline = pd.read_csv("./teamkim_datas.SNS_helpline.csv")
df_naver_symptom = pd.read_csv("./teamkim_datas.SNS_naver_kin_symptom.csv")

df_dcinside["date"] = pd.to_datetime(df_dcinside["date"]).dt.strftime("%Y")
df_helpline["date"] = pd.to_datetime(df_helpline["date"]).dt.strftime("%Y")
df_naver_symptom['question_datetime'] = df_naver_symptom['question_datetime'].str.replace('작성일\\n', '').str.strip()
# "조회수"라는 값이 포함되어있는 행을 모두 지우는 코드
df_naver_symptom = df_naver_symptom[~df_naver_symptom['question_datetime'].str.contains('조회수')]
df_naver_symptom['question_datetime'] = pd.to_datetime(df_naver_symptom['question_datetime']).dt.strftime("%Y")
df_naver_symptom = df_naver_symptom.reset_index()

dc_2020_title = df_dcinside.query("date == '2020'")["title"].to_list()
dc_2020_contents = df_dcinside.query("date == '2020'")["contents"].to_list()
list_dc_2020 = dc_2020_title + dc_2020_contents
helpline_2020_title = df_helpline.query("date == '2020'")["title_disease"].to_list()
helpline_2020_post = df_helpline.query("date == '2020'")["post_contents"].to_list()
helpline_2020_reply = df_helpline.query("date == '2020'")["reply_contents"].to_list()
list_helpline_2020 = helpline_2020_title + helpline_2020_post + helpline_2020_reply
naver_2020_title = df_naver_symptom.query("question_datetime == '2020'")["question_title"].to_list()
naver_2020_question = df_naver_symptom.query("question_datetime == '2020'")["question_content"].to_list()
naver_2020_answer = df_naver_symptom.query("question_datetime == '2020'")["answer1_content"].to_list()
list_naver_2020 = naver_2020_title + naver_2020_question + naver_2020_answer

list_2020 = list_dc_2020 + list_helpline_2020 + list_naver_2020

In [3]:
# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   
            stem=True    
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer, use_idf=True)
features = vectorizer.fit_transform(list_2020)
features.toarray()

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=3, random_state=55) #인스턴스화 #n_components 토픽의 갯수
lda_model.fit(features) #교육

LatentDirichletAllocation(n_components=3, random_state=55)

In [23]:
dictionary_list = vectorizer.get_feature_names_out()

In [24]:
lda_model.components_

array([[0.40362243, 0.38267879, 0.38267879, ..., 0.89528945, 0.33722569,
        0.504349  ],
       [0.33383342, 0.33374114, 0.33374114, ..., 0.3348809 , 1.68072418,
        0.33628225],
       [0.33378843, 0.33366737, 0.33366737, ..., 0.33486625, 0.33541858,
        0.46681943]])

In [25]:
topics_output = lda_model.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [26]:
df_topics_score['dominant_topic_number'] = np.argmax(topics_output, axis=1) #  "topics_output" 배열에서 각 행마다 최대값의 인덱스를 찾아 "dominant_topic_number" 열에 할당

In [27]:
df_topics_score['sentences'] = list_2020

In [28]:
df_topics_score

,0,1,2,dominant_topic_number,sentences
0,0.100947,0.813327,0.085726,1,안락사&존엄사 관련 인권단체를 만들기 위해 카페를 개설했습니다
1,0.092503,0.813245,0.094252,1,저칼륨혈증 주기성 사지마비(feat 꾀병)
2,0.138839,0.139681,0.721480,2,쇼그렌은 언급 1도없네
3,0.104120,0.100845,0.795035,2,유전병 게이들아 드디어 유전자가위가 상용화 된단다
4,0.744131,0.127943,0.127926,0,나보다 희귀한 질병인애 있냐?
...,...,...,...,...,...
417,0.908605,0.045686,0.045710,0,안녕하세요?\n \n온라인 상담실을 통해 문의하신 내용에 대해 답변드립니다.\n\n...
418,0.925751,0.037106,0.037143,0,안녕하세요?\n1:1 상담을 통해 문의하신 문의사항에 대해 답변 드립니다.\n「희귀...
419,0.114555,0.116210,0.769234,2,통풍 초기증상 좋은음식 있나요?
420,0.072452,0.069784,0.857763,2,통풍 초기증상 좋은음식 있나요??\n얼마전 종합건강검진에서 제 남편이 통풍수치가 높...


In [29]:
## 상위 단어 추출
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T #T는 행과 열을 바꿈
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    print(topics_text)
    topics_list.append(topics_text)


topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2']]
df_topics_keywords = pd.DataFrame(topics_list_add)
df_topics_keywords

하다 희귀질환 질환 의료 산정 지원 특례 사업 되다 기준
하다 지방 병원 신경섬유 크론병 수술 투석 소아 이유 체트
하다 병원 되다 섬유화 증후군 척수 오다 군대 싶다 먹다


,0,1,2
0,하다 희귀질환 질환 의료 산정 지원 특례 사업 되다 기준,하다 지방 병원 신경섬유 크론병 수술 투석 소아 이유 체트,하다 병원 되다 섬유화 증후군 척수 오다 군대 싶다 먹다
1,Topic0,Topic1,Topic2
